# Global Temperature Analysis - Exploration
Abinash Patti

### Objective
Analyze global temperature data to identify global warming trends, regional anomolies, and seasonal patterns

### Dataset
Source: Berkeley Earth / Kaggle

Files include temperature readings by city, country, and globally from 1750-2015

In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Load GlobalLandTemperaturesByCountry into a DataFrame
df_global_land_temps_country = pd.read_csv("../data/GlobalLandTemperaturesByCountry.csv")

# Display the info and first few rows
df_global_land_temps_country.info()
df_global_land_temps_country.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577462 entries, 0 to 577461
Data columns (total 4 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   dt                             577462 non-null  object 
 1   AverageTemperature             544811 non-null  float64
 2   AverageTemperatureUncertainty  545550 non-null  float64
 3   Country                        577462 non-null  object 
dtypes: float64(2), object(2)
memory usage: 17.6+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


### Average Temperature Summaries

In [4]:
# Find special summaries
mean_temp = df_global_land_temps_country["AverageTemperature"].mean()
std_temp = df_global_land_temps_country["AverageTemperature"].std()

max_temp = df_global_land_temps_country["AverageTemperature"].max()
min_temp = df_global_land_temps_country["AverageTemperature"].min()

max_temp_row = df_global_land_temps_country[df_global_land_temps_country["AverageTemperature"] == max_temp]
min_temp_row = df_global_land_temps_country[df_global_land_temps_country["AverageTemperature"] == min_temp]

# Display
print("Mean temperature (1750 - 2015): ", round(mean_temp, 2))
print("Standard deviation (1750 - 2015): ", round(std_temp, 2))
print("Highest temperature: ", round(max_temp, 2))
print("Lowest temperature: ", round(min_temp, 2))

Mean temperature (1750 - 2015):  17.19
Standard deviation (1750 - 2015):  10.95
Highest temperature:  38.84
Lowest temperature:  -37.66


In [5]:
# Convert dt col to datetime
df_global_land_temps_country["dt"] = pd.to_datetime(df_global_land_temps_country["dt"])

# Extract year to new column
df_global_land_temps_country["Year"] = df_global_land_temps_country["dt"].dt.year

# Confirm with head
df_global_land_temps_country.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Year
0,1743-11-01,4.384,2.294,Åland,1743
1,1743-12-01,NaN,NaN,Åland,1743
2,1744-01-01,NaN,NaN,Åland,1744
3,1744-02-01,NaN,NaN,Åland,1744
4,1744-03-01,NaN,NaN,Åland,1744


### Extreme Temperatures by Country
Here, we're just getting some rough numbers and later visualizing key statistics. I got a little excited here and did some basic analysis. The data cleaning will be in the following step.

In [11]:
# Group by country
df_country_group = df_global_land_temps_country.groupby("Country")

# List to hold summary
list_summary_extreme_temps = []

# Loop through each country
for country_name, country_group in df_country_group:

    # Drop rows with NaN temperatures
    country_group = country_group.dropna(subset=["AverageTemperature"])

    # Skip if no data
    if country_group.empty:
        continue

    # Hottest and coldest rows
    hottest_row = country_group.loc[country_group["AverageTemperature"].idxmax()]
    coldest_row = country_group.loc[country_group["AverageTemperature"].idxmin()]

    # Extract values
    hottest_temp = hottest_row["AverageTemperature"]
    hottest_year = hottest_row["Year"]
    coldest_temp = coldest_row["AverageTemperature"]
    coldest_year = coldest_row["Year"]

    # Append summary
    list_summary_extreme_temps.append({
        "Country": country_name,
        "HottestYear": hottest_year,
        "HottestTemp": hottest_temp,
        "ColdestYear": coldest_year,
        "ColdestTemp": coldest_temp
    })

# Create DataFrame from summary list
df_summary_extreme_temps = pd.DataFrame(list_summary_extreme_temps)

# Display head
df_summary_extreme_temps.head()


,Country,HottestYear,HottestTemp,ColdestYear,ColdestTemp
0,Afghanistan,1997,28.533,2008,-4.553
1,Africa,2010,27.126,1861,19.523
2,Albania,1757,25.843,1942,-2.049
3,Algeria,2003,35.829,1813,9.526
4,American Samoa,2003,28.543,1915,24.712


### Top 10 Hottest Countries

In [7]:
# Sort by hottest temp descending and display top 10
df_summary_extreme_temps.sort_values(by="HottestTemp", ascending=False).head(10)

,Country,HottestYear,HottestTemp,ColdestYear,ColdestTemp
118,Kuwait,2012,38.842,1911,8.071
228,United Arab Emirates,2010,37.750,1911,15.736
176,Qatar,2012,37.603,1911,13.888
17,Bahrain,2012,37.471,1911,12.160
105,Iraq,2000,37.401,1911,3.838
190,Saudi Arabia,2012,36.495,1992,12.759
3,Algeria,2003,35.829,1813,9.526
134,Mali,2010,35.330,1889,19.059
59,Djibouti,2009,35.175,1918,23.023
163,Oman,2009,35.096,1911,17.563


### Top 10 Coldest Countries

In [8]:
# Sort by coldest temp descending and display top 10
df_summary_extreme_temps.sort_values(by="ColdestTemp").head(10)

,Country,HottestYear,HottestTemp,ColdestYear,ColdestTemp
87,Greenland,2009,0.339,1868,-37.658
57,Denmark,2009,0.699,1868,-36.830
179,Russia,2010,16.893,1838,-30.577
38,Canada,2012,14.796,1911,-28.736
143,Mongolia,2007,20.716,1861,-27.442
114,Kazakhstan,1998,25.562,1969,-23.601
209,Svalbard And Jan Mayen,1761,8.308,1963,-22.587
74,Finland,1757,19.132,1814,-21.200
119,Kyrgyzstan,1956,19.275,1919,-19.161
211,Sweden,1757,17.931,1814,-16.608


In [9]:
# Export summary to CSV
df_summary_extreme_temps.to_csv("../data/analysis/ExtremeTemperaturesByCountry.csv", index=False)

## Data Cleaning

In [22]:
# Check how many nulls in each column
print(df_global_land_temps_country.isnull().sum())

# Check date range
print(df_global_land_temps_country["dt"].min())
print(df_global_land_temps_country["dt"].max())

# Check how many duplicate rows there are
print(df_global_land_temps_country.duplicated().sum())

dt                                   0
AverageTemperature               32651
AverageTemperatureUncertainty    31912
Country                              0
Year                                 0
dtype: int64
1743-11-01 00:00:00
2013-09-01 00:00:00
0


- The AverageTemperature and AverageTemperatureUncertainty columns have a lot of null values, as can be expected since the dataset includes recordings from very old dates.
- The date range is 1743-2013, slightly different from the 1750-2015 we originally thought. 
- No duplicate rows at all, which is great.

In [23]:
# Create a clean DataFrame
df_global_land_temps_country_clean = df_global_land_temps_country.copy()

# Drop NaN values
df_global_land_temps_country_clean = df_global_land_temps_country_clean.dropna(subset=["AverageTemperature", "AverageTemperatureUncertainty"])

# Convert date column to datetime
df_global_land_temps_country_clean["dt"] = pd.to_datetime(df_global_land_temps_country_clean["dt"])

# Check the rows of clean DataFrame vs original
print("Rows removed in clean DataFrame: ", len(df_global_land_temps_country) - len(df_global_land_temps_country_clean))

# Check if dt is in datetime format
print(df_global_land_temps_country_clean["dt"].dtype)

# Display head of clean DataFrame
print(df_global_land_temps_country_clean.head())

Rows removed in clean DataFrame:  32651
datetime64[ns]
          dt  AverageTemperature  AverageTemperatureUncertainty Country  Year
0 1743-11-01               4.384                          2.294   Åland  1743
5 1744-04-01               1.530                          4.680   Åland  1744
6 1744-05-01               6.702                          1.789   Åland  1744
7 1744-06-01              11.609                          1.577   Åland  1744
8 1744-07-01              15.342                          1.410   Åland  1744


I removed 32651 rows by dropping all rows with a null Average Temperature and AverageTemperatureUncertainty. Those rows are not helpful in the context of this analysis, and the dataset remains massive even after this cleaning. Additionally, I converted the dt column to the datetime format, which is the standard and will help me in my analysis.